In [ ]:
import pandas as pd
df=pd.read_excel(r'C:\Users\loges\Documents\GitHub\ML-App\data\features\final_data.xlsx')

In [58]:
df=df[['ft','bt','km','oem','transmission','ownerNo','modelYear','Engine Displacement',"Seats","price"]]

In [59]:
df

,ft,bt,km,oem,transmission,ownerNo,modelYear,Engine Displacement,Seats,price
0,1,7,377,13,0,1,2022,1493.0,5.0,19.00
2,4,7,3472,21,0,1,2020,1991.0,5.0,56.00
9,4,2,2984,20,1,1,2019,1197.0,5.0,6.63
10,4,2,2177,8,0,1,2014,1198.0,5.0,3.90
11,1,7,4026,24,1,1,2019,1461.0,5.0,7.84
...,...,...,...,...,...,...,...,...,...,...
8362,4,2,326,20,1,1,2002,796.0,5.0,40000.00
8364,4,2,237,2,1,1,2011,1199.0,5.0,1.08
8365,4,2,3112,20,1,1,2007,796.0,5.0,1.05
8366,4,7,766,27,1,1,2022,999.0,5.0,8.50


In [7]:
df=df[['ft','bt','km','oem','transmission','ownerNo','modelYear','Engine Displacement',"Seats","price"]]

In [60]:
from sklearn.model_selection import train_test_split,GridSearchCV
x=df.drop('price',axis=1)
y=df['price']

In [61]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [65]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
def model_train(x_train,x_test,y_train,y_test):


            models = {
                'Linear Regression': LinearRegression(),
                'Random Forest Regressor': RandomForestRegressor(random_state=42),
                'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
                'XGBoost Regressor': XGBRegressor(random_state=42, verbosity=0)
            }


            param_grids = {
                'Linear Regression': {
                    'fit_intercept': [True, False],
                    'positive': [True, False]
                },
                'Random Forest Regressor': {
                    'n_estimators': [100, 200],
                    'max_depth': [None, 10],
                    'min_samples_split': [2, 5],
                    'min_samples_leaf': [1, 2],
                    'max_features': ['sqrt']
                },
                'Gradient Boosting Regressor': {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1],
                    'max_depth': [3, 5],
                    'min_samples_split': [2, 5],
                    'min_samples_leaf': [1, 2],
                    'subsample': [0.8, 1.0],
                    'max_features': ['sqrt']
                },
                'XGBoost Regressor': {
                    'n_estimators': [100, 200],
                    'learning_rate': [0.05, 0.1],
                    'max_depth': [3, 5],
                    'subsample': [0.8, 1.0],
                    'colsample_bytree': [0.8, 1.0],
                    'gamma': [0, 1],
                    'reg_alpha': [0, 0.1],
                    'reg_lambda': [1, 2]
                }
            }


            results = []

            for name, model in models.items():
                print(f"\n🔍 Running GridSearchCV for {name}...")
                grid = GridSearchCV(
                    estimator=model,
                    param_grid=param_grids[name],
                    cv=3,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    verbose=0
                )
                grid.fit(x_train, y_train)

                best_model = grid.best_estimator_
                y_pred = best_model.predict(x_test)

                mse = mean_squared_error(y_test, y_pred)
                r2 = r2_score(y_test, y_pred)
                mae=mean_absolute_error(y_test,y_pred)

                results.append({
                    "Model": name,
                    "Best Params": grid.best_params_,
                    "CV Score (neg MSE)": grid.best_score_,
                    "Test MSE": mse,
                    "Test R2": r2
                })

            # 5. Display results as DataFrame
            results_df = pd.DataFrame(results)
            return results_df

In [67]:
model_df=model_train(x_train,x_test,y_train,y_test)


🔍 Running GridSearchCV for Linear Regression...

🔍 Running GridSearchCV for Random Forest Regressor...

🔍 Running GridSearchCV for Gradient Boosting Regressor...

🔍 Running GridSearchCV for XGBoost Regressor...

✅ GridSearch Results:
                         Model  \
0            Linear Regression   
1      Random Forest Regressor   
2  Gradient Boosting Regressor   
3            XGBoost Regressor   

                                         Best Params  CV Score (neg MSE)  \
0         {'fit_intercept': True, 'positive': False}       -7.102498e+07   
1  {'max_depth': None, 'max_features': 'sqrt', 'm...       -5.043237e+07   
2  {'learning_rate': 0.05, 'max_depth': 5, 'max_f...       -5.087656e+07   
3  {'colsample_bytree': 0.8, 'gamma': 0, 'learnin...       -5.341545e+07   

       Test MSE   Test R2  
0  8.063931e+07  0.096419  
1  6.419626e+07  0.280667  
2  5.189815e+07  0.418470  
3  5.008741e+07  0.438760  


In [68]:
model_df

,Model,Best Params,CV Score (neg MSE),Test MSE,Test R2
0,Linear Regression,"{'fit_intercept': True, 'positive': False}",-7.102498e+07,8.063931e+07,0.096419
1,Random Forest Regressor,"{'max_depth': None, 'max_features': 'sqrt', 'm...",-5.043237e+07,6.419626e+07,0.280667
2,Gradient Boosting Regressor,"{'learning_rate': 0.05, 'max_depth': 5, 'max_f...",-5.087656e+07,5.189815e+07,0.418470
3,XGBoost Regressor,"{'colsample_bytree': 0.8, 'gamma': 0, 'learnin...",-5.341545e+07,5.008741e+07,0.438760


In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor

# Load regression dataset
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Base regressors
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('ridge', RidgeCV())
]

# Meta-model (final estimator)
meta_model = LinearRegression()

# Stacking Regressor
stack_model = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

# Train
stack_model.fit(X_train, y_train)

# Evaluate
print("R^2 on test set:", stack_model.score(X_test, y_test))


In [76]:
model_df[model_df['Model']=='XGBoost Regressor']['Best Params'].values

array([{'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 1, 'subsample': 0.8}],
      dtype=object)

In [96]:
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor

xgb= XGBRegressor(colsample_bytree= 0.8, gamma= 0, learning_rate= 0.05,
                  max_depth= 3, n_estimators= 100, reg_alpha=0.1,
                  reg_lambda = 1, subsample= 0.8)
y_pred=xgb.fit(x_train,y_train).predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae=mean_absolute_error(y_test,y_pred)

In [100]:
y_pred=xgb.fit(x_train,y_train).predict(x_test)
#y_pred=bag_xgb.predict(x_test)

In [101]:
r2_score(y_test,y_pred)

0.43685915889717253

In [111]:
import pickle

# Save model
with open("/content/xgb_model.pkl", "wb") as f:
    pickle.dump(xgb, f)

In [113]:
loaded_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [114]:
from google.colab import files

# Download the file
files.download("/content/xgb_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>